# Introdução

Esse caderno tem por objetivo a criação de um modelo básico de treinamento, utilizando o classificador [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), que faça a predição de homologação de arquivamentos de procedimentos enviados à 1A.CAM do MPF.

Nesse modelo vamos passar a considerar os textos das íntegras.

A primeira passagem, serão criadas colunas com as palavras-chaves informadas pela área negocial que vão indicar a ausência ou presença do termo no texto da íntegra da peça de promoção do arquivamento.

Na segunda passagem, vamos passar a considerar unicamente o texto da íntegra.

**Nota**: os dados desse modelo foram recuperados de procedimentos que tiveram suas deliberações realizadas após o dia 02/07/2018, data em que a nova composição tomou posse na 1A.CAM.

# Contagem de termos mais frequentes

Vamos fazer uma análise dos textos das íntegras e tentar encontrar os termos mais frequentes.

O objetivo é:

- fazer uma confirmação das frequências dos termos com a lista de palavras-chaves que nos foi enviada;
- descobrir outras palavras que possam ser indicativos de homologação ou não.

Serão realizadas três análises:

- considerando todas as íntegras;
- considerando apenas as íntegras das peças que tiveram seu homologamento confirmado;
- considerando os não homologados;

Dessa análise, podemos criar colunas derivadas.

# Carga de dados e pré-processamento

Vamos fazer a carga dos dados.

Vamos ler os textos das íntegras, limpá-los e associar ao dataframe com os dados de homologação.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

from sklearn import metrics

In [3]:
PATH = "../data/"

In [4]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andrethiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/andrethiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
# Removendo Stop words

from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = stopwords.words('portuguese')
stop_words.extend(['n', 'nº', 'n°', 'n.º', ',', '.', '!', '?', ';', ':', '...', 'º', '–', '/', '(', ')'])

def get_text(file):
    with open(file, encoding='utf-8') as f:
        data=' '.join(line.strip() for line in f)
        f.close()
    
    return data.strip()

def get_text_non_stop_words(text):
    filtered_text = ''
    for w in word_tokenize(text, language='portuguese'):
        if w not in stop_words:
            filtered_text += w
            filtered_text += ' '            
    return filtered_text.strip()   
    

folder_integras = f'{PATH}/integras-textos'

#onlyfiles = [f for f in listdir(folder_integras) if isfile(join(folder_integras, f))]

texts = {}
for file in listdir(folder_integras):
    if isfile(join(folder_integras, file)):
        texts[file.split('.')[0]] = get_text_non_stop_words(get_text(join(folder_integras,file)))

    
texts['76397008']

'Procedimento Administrativo Ministério Público Federal Procuradoria República Ceará Inquérito Civil n. 1.15.000.000424/2017-42 Promoção Arquivamento 938/2017/GAB-ANTJ/PR/CE PROMOÇÃO DE ARQUIVAMENTO Trata-se Inquérito Civil instaurado partir manifestação sigilosa aluno Colégio Polícia Militar Ceará informando alunos estariam sendo forçados permanecer sala aula aparelho ar condicionado quebrado geraria intenso desconforto alunos chegando fazê-los passar mal Chamado manifestar-se comandante Colégio Polícia Militar Ceará informou Colégio Polícia Militar Ceará dispõe 35 salas 11 ar condicionado 24 ventiladores Em manifestação comandante confirmou 05 salas possuem ar condicionado encontram-se aparelhos danificados porém informou tais salas dispõem ventiladores horários calor intenso professores liberdade ministrar aulas pátio alunos usar uniforme leves Visando investigar procedência reclamações justificativas apresentadas solicitado Seção Segurança Orgânica Transporte desta PR/CE realizasse

In [17]:
# Ver se dá pra usar o sklearn aqui...

from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(ngram_range=(1,6))
bag = count.fit_transform(list(texts.values()))

In [33]:
filtered_text = [w for w in word_tokens if not w in stop]

In [23]:
from heapq import nlargest

top_ten = nlargest(10, count.vocabulary_, key=count.vocabulary_.get)

print(top_ten)

['üá qep 7ô w640 tej 61', 'üá qep 7ô w640 tej', 'üá qep 7ô w640', 'üá qep 7ô', 'üá qep', 'üá', 'üsuc ogrto oa rei jc1 iii', 'üsuc ogrto oa rei jc1', 'üsuc ogrto oa rei', 'üsuc ogrto oa']
